In [8]:
import spark_catalog
import duckdb

In [9]:
catalog = spark_catalog.SparkCatalog(app_name="kexp", lake_location="../../../data/spark/kexp")

In [10]:
kexp_playlist = catalog.get_data_frame("bronze", "KEXP_PLAYLIST")

In [11]:
kexp_playlist.show()

23/04/10 12:43:15 WARN DeltaLog: Change in the table id detected while updating snapshot. 
Previous snapshot = Snapshot(path=file:/Users/timburns/Documents/PythonProjects/bigdataplatforms/data/spark/kexp/bronze/KEXP_PLAYLIST/_delta_log, version=1, metadata=Metadata(a3805b47-51d9-439d-8e14-902a2005a717,null,null,Format(parquet,Map()),{"type":"struct","fields":[{"name":"airdate","type":"timestamp","nullable":true,"metadata":{}},{"name":"album","type":"string","nullable":true,"metadata":{}},{"name":"artist","type":"string","nullable":true,"metadata":{}},{"name":"artist_ids","type":{"type":"array","elementType":"string","containsNull":true},"nullable":true,"metadata":{}},{"name":"comment","type":"string","nullable":true,"metadata":{}},{"name":"id","type":"integer","nullable":true,"metadata":{}},{"name":"image_uri","type":"string","nullable":true,"metadata":{}},{"name":"is_live","type":"boolean","nullable":true,"metadata":{}},{"name":"is_local","type":"boolean","nullable":true,"metadata":{}

+-------------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+-------+--------+----------+--------------------+--------------------+---------+------------+------------+----------------+----------+---------------+-----+--------------------+--------------------+--------------------+--------+--------------------+
|            airdate|               album|              artist|          artist_ids|             comment|     id|           image_uri|is_live|is_local|is_request|           label_ids|              labels|play_type|recording_id|release_date|release_group_id|release_id|rotation_status| show|            show_uri|                song|       thumbnail_uri|track_id|                 uri|
+-------------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+-------+--------+----------+--------------------+--------------------+---------+------------+-----

In [12]:
catalog.sql("select count(*) as play_count from KEXP_PLAYLIST").show()

+----------+
|play_count|
+----------+
|      1563|
+----------+



In [15]:
catalog.sql("select id, airdate, artist, song, show from KEXP_PLAYLIST order by id desc").show()

+-------+-------------------+--------------------+--------------------+-----+
|     id|            airdate|              artist|                song| show|
+-------+-------------------+--------------------+--------------------+-----+
|3178592|2023-04-10 12:31:15|  The Murder Capital|      For Everything|56664|
|3178591|2023-04-10 12:31:10|                null|                null|56664|
|3178590|2023-04-10 12:25:31|      Fontaines D.C.|         ’Cello Song|56664|
|3178589|2023-04-10 12:20:21|            Adorable|      Sunshine Smile|56664|
|3178588|2023-04-10 12:17:46|               Bully|      Days Move Slow|56664|
|3178587|2023-04-10 12:16:19|                null|                null|56664|
|3178586|2023-04-10 12:10:33|De La Soul feat. ...|               Buddy|56664|
|3178585|2023-04-10 12:06:56|           Deee‐Lite|Groove Is in the ...|56664|
|3178584|2023-04-10 12:02:50|     Jungle Brothers|  Doin’ Our Own Dang|56664|
|3178583|2023-04-10 12:01:53|                null|              

In [16]:
catalog.get_data_frame("bronze", "KEXP_PLAYLIST").printSchema()

root
 |-- airdate: timestamp (nullable = true)
 |-- album: string (nullable = true)
 |-- artist: string (nullable = true)
 |-- artist_ids: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- comment: string (nullable = true)
 |-- id: integer (nullable = true)
 |-- image_uri: string (nullable = true)
 |-- is_live: boolean (nullable = true)
 |-- is_local: boolean (nullable = true)
 |-- is_request: boolean (nullable = true)
 |-- label_ids: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- labels: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- play_type: string (nullable = true)
 |-- recording_id: integer (nullable = true)
 |-- release_date: date (nullable = true)
 |-- release_group_id: integer (nullable = true)
 |-- release_id: integer (nullable = true)
 |-- rotation_status: string (nullable = true)
 |-- show: integer (nullable = true)
 |-- show_uri: string (nullable = true)
 |-- song: string (nullable = tru

In [18]:
catalog.sql("select id, size(artist_ids) from KEXP_PLAYLIST order by size(artist_ids) desc").show()

+-------+----------------+
|     id|size(artist_ids)|
+-------+----------------+
|3177634|               8|
|3177449|               7|
|3177284|               7|
|3177633|               6|
|3178261|               6|
|3177629|               5|
|3177450|               5|
|3177516|               4|
|3178533|               4|
|3178513|               4|
|3178505|               3|
|3177439|               3|
|3178085|               3|
|3178518|               3|
|3177162|               3|
|3177134|               3|
|3178586|               3|
|3178292|               3|
|3177267|               3|
|3177381|               3|
+-------+----------------+
only showing top 20 rows



In [27]:
catalog.sql("select artist, song, album, artist_ids from KEXP_PLAYLIST where ID = 3177634").show(truncate=False, vertical=True)

-RECORD 0----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 artist     | 9umba, Toss & Mdoovar feat. Sir Trill, Sino Msolo, Lady Du, Young Stunna & Slade                                                                                                                                                                                                                                 
 song       | uMlando                                                                                                                                                                                                                                                                                                          
 album      | uMlando                   

In [28]:
kexp_playlist = catalog.get_data_frame("bronze", "KEXP_PLAYLIST")

In [31]:
from pyspark.sql.functions import explode

In [32]:
kexp_playlist.withColumn("artist_ids", explode(kexp_playlist.artist_ids)).show()

+-------------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+-------+--------+----------+--------------------+--------------------+---------+------------+------------+----------------+----------+---------------+-----+--------------------+--------------------+--------------------+--------+--------------------+
|            airdate|               album|              artist|          artist_ids|             comment|     id|           image_uri|is_live|is_local|is_request|           label_ids|              labels|play_type|recording_id|release_date|release_group_id|release_id|rotation_status| show|            show_uri|                song|       thumbnail_uri|track_id|                 uri|
+-------------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+-------+--------+----------+--------------------+--------------------+---------+------------+-----

In [36]:
artist_df = catalog.sql("select artist, artist_ids from KEXP_PLAYLIST order by artist")

In [37]:
artist_df.withColumn("artist_ids", explode(artist_df.artist_ids)).show()

+--------------------+--------------------+
|              artist|          artist_ids|
+--------------------+--------------------+
|     $wingin’ Utter$|25302723-b24f-4d0...|
|                 310|5e29d25a-ba6f-402...|
|            38 Spesh|3e63ef4f-93c5-48a...|
|               6LACK|07832b42-8826-4ab...|
|           79rs Gang|9d7c95f4-9575-4b7...|
|            8fatfat8|030c731f-6b7c-407...|
|            8fatfat8|030c731f-6b7c-407...|
|9th Wonder feat. ...|48513f24-37d5-423...|
|9th Wonder feat. ...|6b1acf37-0b3b-4cc...|
|9th Wonder feat. ...|6705519c-4f9b-4e5...|
|9umba, Toss & Mdo...|d3253aba-df21-4dc...|
|9umba, Toss & Mdo...|13bcba23-2fd2-4b7...|
|9umba, Toss & Mdo...|936c3773-6467-486...|
|9umba, Toss & Mdo...|7a7b5cfe-24e0-42f...|
|9umba, Toss & Mdo...|40b8dcd1-6efd-49f...|
|9umba, Toss & Mdo...|8d3dd2fe-0aac-4a3...|
|9umba, Toss & Mdo...|e701b534-eea5-4b6...|
|9umba, Toss & Mdo...|7c745985-26bf-444...|
|             A House|c2f465fc-1f81-4d2...|
|A Tribe Called Quest|9689aa5a-4